In [11]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [12]:
filename='ReportTester-4000009439.xlsx'

In [13]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data.columns = columns
    balance = data['Beneficio'].iloc[start_index+2]
    data = data.iloc[start_index+3:end_index-1,:]
    resultados = []
    for i in range(0, len(data) - 1, 2):
        fila_in = data.iloc[i]
        fila_out = data.iloc[i + 1]
        if fila_in['Dirección'] == 'in' and fila_out['Dirección'] == 'out':
            precio_entrada = fila_in['Precio']
            precio_salida = fila_out['Precio']
            beneficio = fila_out['Beneficio'] - abs(fila_in['Comisión']) - abs(fila_out['Comisión'])
            balance += beneficio
            resultados.append({
                'Fecha': fila_in['Fecha/Hora'],
                'Símbolo': fila_in['Símbolo'],
                'Tipo': fila_in['Tipo'],
                'Precio Entrada': precio_entrada,
                'Precio Salida': precio_salida,
                'Volumen': fila_in['Volumen '],
                'Beneficio': beneficio,
                'Balance': balance
            })
    return pd.DataFrame(resultados)

In [14]:
# Statistics
df = load_xls_report(filename)
total_trades = df.shape[0]
profit_trades = df[df['Beneficio']>0].shape[0]
profit_avg = df[df['Beneficio']>0]['Beneficio'].mean()
loss_trades = df[df['Beneficio']<0].shape[0]
loss_avg = df[df['Beneficio']<0]['Beneficio'].mean()
beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
total_trades = profit_trades + loss_trades
expected_profit = beneficio_neto / total_trades
profit_avg_points = abs(df[df['Beneficio']>0]['Precio Entrada'] - df[df['Beneficio']>0]['Precio Salida']).mean() / 0.1
loss_avg_points = abs(df[df['Beneficio']<0]['Precio Entrada'] - df[df['Beneficio']<0]['Precio Salida']).mean() / 0.1
punto_equilibrio = abs(loss_avg)/(abs(loss_avg)+profit_avg)
win_rates = profit_trades/total_trades
balances = df['Balance']
cummax_balance = balances.cummax()
drawdowns = cummax_balance - balances
drawdown_percentages = drawdowns / cummax_balance
max_drawdown_percentage = drawdown_percentages.max() * 100

In [15]:
print(f"Número de operaciones totales: {total_trades}")
print(f"Número de operaciones ganadoras: {profit_trades}")
print(f"Número de operaciones perdedoras: {loss_trades}")
print(f"Media de puntos en operaciones ganadoras {profit_avg_points:.2f} puntos")
print(f"Media de puntos en operaciones perdedoras {loss_avg_points:.2f} puntos")
print(f"Ganancia media: {profit_avg:.2f}€")
print(f"Pérdida media: {loss_avg:.2f}€")
print(f"Rentabilidad esperada: {expected_profit:.4f}€")
print(f"Porcentaje de aciertos actual: {win_rates*100.0:.2f}%")
print(f"Porcentaje de aciertos para equilibrio: {punto_equilibrio*100.0:.2f}%")
print(f"El máximo drawdown es: {max_drawdown_percentage:.2f}%")

Número de operaciones totales: 20906
Número de operaciones ganadoras: 16202
Número de operaciones perdedoras: 4704
Media de puntos en operaciones ganadoras 82.62 puntos
Media de puntos en operaciones perdedoras 241.71 puntos
Ganancia media: 0.77€
Pérdida media: -2.47€
Rentabilidad esperada: 0.0371€
Porcentaje de aciertos actual: 77.50%
Porcentaje de aciertos para equilibrio: 76.35%
El máximo drawdown es: 6.54%
